<a href="https://colab.research.google.com/github/mkuczynski11/micropython-tflite/blob/model_creation_update/big_schroom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports



In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

### Setup training and validation datasets

In [2]:
batch_size = 32
img_height = 96
img_width = 96
# Use ready cnn architecture to detect image features
# cnn_feature_vector = "https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/feature_vector/5"
# cnn_feature_vector = "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5"
# cnn_feature_vector = "https://tfhub.dev/google/imagenet/mobilenet_v1_025_224/feature_vector/5"
cnn_feature_vector = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/5"
quantization = 'float' # Possible ['none', 'dynamic', 'float', 'int']

In [3]:
# Usage of google drive where we are storing our data
from google.colab import drive
drive.mount('/content/gdrive')

# data_dir = '/content/gdrive/MyDrive/Mushrooms' # directory of your data
# !rm -rf /content/gdrive/MyDrive/Mushrooms/.ipynb_checkpoints/

# data_dir = '/content/gdrive/MyDrive/flowers' # directory of your data
# !rm -rf /content/gdrive/MyDrive/flowers/.ipynb_checkpoints/

data_dir = '/content/gdrive/MyDrive/vehicles' # directory of your data
!rm -rf /content/gdrive/MyDrive/vehicles/.ipynb_checkpoints/

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Mounted at /content/gdrive
Found 28045 files belonging to 17 classes.
Using 22436 files for training.


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 28045 files belonging to 17 classes.
Using 5609 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['Ambulance', 'Barge', 'Bicycle', 'Boat', 'Bus', 'Car', 'Cart', 'Caterpillar', 'Helicopter', 'Limousine', 'Motorcycle', 'Segway', 'Snowmobile', 'Tank', 'Taxi', 'Truck', 'Van']


### Define model

In [6]:
import tensorflow_hub as hub

num_classes = len(class_names)

# Name model with used architecture
model_name = cnn_feature_vector.split('/')[5]
print(model_name)
model_dir = pathlib.Path(f'/tmp/models/{model_name}')
model_dir.mkdir(exist_ok=True, parents=True)
print(model_dir)

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    hub.KerasLayer(cnn_feature_vector, trainable=False),
    tf.keras.layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

mobilenet_v2_100_96
/tmp/models/mobilenet_v2_100_96


### Train model

In [7]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)
# Also add epochs that model has been trained on
import sys
original_stdout = sys.stdout # Save a reference to the original standard output
with open(f'{model_dir}/{model_name}.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    model.summary()
    sys.stdout = original_stdout # Reset the standard output to its original value

Epoch 1/20
702/702 [==============================] - 9028s 13s/step - loss: 0.6020 - accuracy: 0.8144 - val_loss: 0.4790 - val_accuracy: 0.8536
Epoch 2/20
702/702 [==============================] - 379s 535ms/step - loss: 0.3950 - accuracy: 0.8695 - val_loss: 0.4635 - val_accuracy: 0.8577
Epoch 3/20
702/702 [==============================] - 382s 540ms/step - loss: 0.3358 - accuracy: 0.8865 - val_loss: 0.4722 - val_accuracy: 0.8584
Epoch 4/20
702/702 [==============================] - 388s 550ms/step - loss: 0.2985 - accuracy: 0.8988 - val_loss: 0.4874 - val_accuracy: 0.8533
Epoch 5/20
702/702 [==============================] - 397s 561ms/step - loss: 0.2772 - accuracy: 0.9052 - val_loss: 0.5008 - val_accuracy: 0.8495
Epoch 6/20
702/702 [==============================] - 985s 1s/step - loss: 0.2532 - accuracy: 0.9131 - val_loss: 0.5107 - val_accuracy: 0.8481
Epoch 7/20
702/702 [==============================] - 948s 1s/step - loss: 0.2389 - accuracy: 0.9186 - val_loss: 0.5374 - val_ac

### Convert to tflite

In [8]:
def representative_data_gen():
  a = []
  limit = 3
  counter = 0
  for batch in train_ds:
    counter += 1
    if counter == 3:
      break
    input_tensor = batch[0]
    for i in range(input_tensor.shape[0]):
      a.append(input_tensor[i])
  a = np.asarray(a)
  ds = tf.data.Dataset.from_tensor_slices(a).batch(1)
  for i in ds.take(100):
    yield [i]

def write_to_file(text):
  with open(f'{model_dir}/{model_name}.txt', 'a') as f:
    f.write(text + '\n')

def no_quantization(converter):
  write_to_file('quantization: none')
  return converter.convert()

# Smaller model with quantized weights, but other variable data still in float format
# NOTE: Hybrid models are not supported in TFlite Micro
def dynamic_quantization(converter):
  write_to_file('quantization: dynamic')
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  return converter.convert()

# All weights and variable data quantized
# Not compatible with devices that perform only integer-based operations 
def float_quantization(converter):
  write_to_file('quantization: float')
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.representative_dataset = representative_data_gen
  return converter.convert()

# All weights and variable data quantized
# Internal quantization is the same as with float, but inuput and output tensors format is changed to uint8
def int_quantization(converter):
  write_to_file('quantization: int')
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.representative_dataset = representative_data_gen
  # Ensure that if any ops can't be quantized, the converter throws an error
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
  # Set the input and output tensors to uint8 (APIs added in r2.3)
  converter.inference_input_type = tf.uint8
  converter.inference_output_type = tf.uint8
  return converter.convert()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
if quantization == 'none':
  tflite_model = no_quantization(converter)
elif quantization == 'dynamic':
  tflite_model = dynamic_quantization(converter)
elif quantization == 'float':
  tflite_model = float_quantization(converter)
elif quantization == 'int':
  tflite_model = int_quantization(converter)

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


### Check model accuracy

In [9]:
loss, acc = model.evaluate(val_ds)
write_to_file(f'loss={loss}, accuracy={acc}')

176/176 [==============================] - 79s 440ms/step - loss: 0.7322 - accuracy: 0.8285


### Save model

In [10]:
import pathlib

model_save_name = f'{model_name}.tflite'

tflite_model_file = model_dir/model_save_name
tflite_model_file.write_bytes(tflite_model)

with open(f'{model_dir}/labels.txt', 'a') as f:
    for class_name in train_ds.class_names:
      f.write(class_name + '\n')


### Test model

In [1]:
# # Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

# Load image
import urllib.request
import numpy as np
from PIL import Image

urllib.request.urlretrieve(
  'https://www.autofromchina.com/u_file/2208/products/05/17e85350c7.jpg.240x240.jpg',
   "gfg.jpg")
image = Image.open('gfg.jpg').resize((96,96))
image = np.asarray(image)
image = image.reshape((96, 96, 3))

test_image = np.expand_dims(image, axis=0).astype(input_details["dtype"])
interpreter.set_tensor(input_details['index'], test_image)
interpreter.invoke()

output = interpreter.get_tensor(output_details['index'])[0]
print(output)
print(output.argmax())

import matplotlib.pylab as plt

plt.imshow(image)

im = Image.fromarray(image)
im.save('veh.jpg')